In [1]:
import geopandas as gpd
import pandas as pd

## Open Counties data

In [2]:
file_path = "data/cb_2024_us_county_500k/cb_2024_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)

## Get Ethnicity amounts

In [3]:
acs_ethnicity = pd.read_csv(
    "data/ACSDT5Y2023.B04006_2025-05-02T185520/ACSDT5Y2023.B04006-Data.csv",
    low_memory=False,
)

#### Get Column names

In [4]:
acs_column_names = pd.read_csv(
    "data/ACSDT5Y2023.B04006_2025-05-02T185520/ACSDT5Y2023.B04006-Column-Metadata.csv"
)

In [5]:
ethnicity_cols = list(
    acs_column_names[
        (acs_column_names["Column Name"] == "B04006_051E")
        | (acs_column_names["Column Name"] == "B04006_049E")
    ]["Column Name"]
)

#### Back to Data

In [6]:
ethnicity_per_county_df = acs_ethnicity[["GEO_ID", *ethnicity_cols]]
ethnicity_per_county_df.columns = ethnicity_per_county_df.iloc[0]
ethnicity_per_county_df = ethnicity_per_county_df[1:]
ethnicity_per_county_df = ethnicity_per_county_df.rename(
    columns={
        "Estimate!!Total:!!Italian": "ITALIAN",
        "Estimate!!Total:!!Irish": "IRISH",
        "Geography": "GEOIDFQ",
    }
)

## Merge Data

In [7]:
ethnicity_per_county_gdf = counties_gdf.merge(
    ethnicity_per_county_df, on="GEOIDFQ", how="left"
)

In [8]:
ethnicity_per_county_gdf["county_id"] = ethnicity_per_county_gdf["GEOIDFQ"].str[9:]
ethnicity_per_county_gdf["state_id"] = ethnicity_per_county_gdf["GEOIDFQ"].str[9:11]

In [9]:
ethnicity_per_county_gdf = ethnicity_per_county_gdf.fillna(0)
ethnicity_per_county_gdf["ITALIAN"] = ethnicity_per_county_gdf["ITALIAN"].astype(int)
ethnicity_per_county_gdf["IRISH"] = ethnicity_per_county_gdf["IRISH"].astype(int)

In [10]:
def create_label(row) -> str:
    italian = row["ITALIAN"]
    irish = row["IRISH"]
    if italian == irish and italian == 0:
        return "No Reported Ancestry"
    if italian == irish:
        return "Same Amount"
    if italian > irish:
        return "More Italian"
    if italian < irish:
        return "More Irish"
    return "Edge Case"

In [11]:
ethnicity_per_county_gdf["label"] = ethnicity_per_county_gdf.apply(
    lambda row: create_label(row), axis=1
)

In [12]:
ethnicity_per_county_gdf[ethnicity_per_county_gdf["label"] == "Edge Case"]

,STATEFP,COUNTYFP,COUNTYNS,GEOIDFQ,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry,IRISH,ITALIAN,county_id,state_id,label


In [13]:
ethnicity_per_county_gdf = ethnicity_per_county_gdf.to_crs(9311)
ethnicity_per_county_gdf.to_file("data/italian_vs_irish_county.gpkg")